<a href="https://colab.research.google.com/github/Siskanti/SqlAlcheny-ORM/blob/main/SqlALchemy_ORM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NAMA : SISKANTI

NIM : H071201064

PRODI : SISTEM INFORMASI

TEKNOLOGI BASIS DATA

SQLALCHEMY ORM

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd

In [ ]:
import sqlalchemy as db
from sqlalchemy import *
from sqlalchemy.orm import declarative_base, sessionmaker, relationship
from sqlalchemy.orm import relationship
from sqlalchemy import Column, DECIMAL, Date, ForeignKey, String, Text
from sqlalchemy.dialects.mysql import INTEGER, MEDIUMBLOB, MEDIUMTEXT, SMALLINT

In [ ]:
engine=create_engine('mysql://root@localhost:3306/classicmodels')
Base = declarative_base()
Session = sessionmaker(bind = engine)
session = Session()

In [ ]:
class Office(Base):
    __tablename__ = 'offices'

    officeCode = Column(String(10), primary_key=True)
    city = Column(String(50), nullable=False)
    phone = Column(String(50), nullable=False)
    addressLine1 = Column(String(50), nullable=False)
    addressLine2 = Column(String(50))
    state = Column(String(50))
    country = Column(String(50), nullable=False)
    postalCode = Column(String(15), nullable=False)
    territory = Column(String(10), nullable=False)

In [ ]:
class Productline(Base):
    __tablename__ = 'productlines'

    productLine = Column(String(50), primary_key=True)
    textDescription = Column(String(4000))
    htmlDescription = Column(MEDIUMTEXT)
    image = Column(MEDIUMBLOB)

In [ ]:
class Employee(Base):
    __tablename__ = 'employees'

    employeeNumber = Column(INTEGER(11), primary_key=True)
    lastName = Column(String(50), nullable=False)
    firstName = Column(String(50), nullable=False)
    extension = Column(String(10), nullable=False)
    email = Column(String(100), nullable=False)
    officeCode = Column(ForeignKey('offices.officeCode'), nullable=False, index=True)
    reportsTo = Column(ForeignKey('employees.employeeNumber'), index=True)
    jobTitle = Column(String(50), nullable=False)

    office = relationship('Office')
    parent = relationship('Employee', remote_side=[employeeNumber])

In [ ]:
class Product(Base):
    __tablename__ = 'products'

    productCode = Column(String(15), primary_key=True)
    productName = Column(String(70), nullable=False)
    productLine = Column(ForeignKey('productlines.productLine'), nullable=False, index=True)
    productScale = Column(String(10), nullable=False)
    productVendor = Column(String(50), nullable=False)
    productDescription = Column(Text, nullable=False)
    quantityInStock = Column(SMALLINT(6), nullable=False)
    buyPrice = Column(DECIMAL(10, 2), nullable=False)
    MSRP = Column(DECIMAL(10, 2), nullable=False)

    productline = relationship('Productline')

In [ ]:
class Customer(Base):
    __tablename__ = 'customers'

    customerNumber = Column(INTEGER(11), primary_key=True)
    customerName = Column(String(50), nullable=False)
    contactLastName = Column(String(50), nullable=False)
    contactFirstName = Column(String(50), nullable=False)
    phone = Column(String(50), nullable=False)
    addressLine1 = Column(String(50), nullable=False)
    addressLine2 = Column(String(50))
    city = Column(String(50), nullable=False)
    state = Column(String(50))
    postalCode = Column(String(15))
    country = Column(String(50), nullable=False)
    salesRepEmployeeNumber = Column(ForeignKey('employees.employeeNumber'), index=True)
    creditLimit = Column(DECIMAL(10, 2))

    employee = relationship('Employee')

In [ ]:
class Order(Base):
    __tablename__ = 'orders'

    orderNumber = Column(INTEGER(11), primary_key=True)
    orderDate = Column(Date, nullable=False)
    requiredDate = Column(Date, nullable=False)
    shippedDate = Column(Date)
    status = Column(String(15), nullable=False)
    comments = Column(Text)
    customerNumber = Column(ForeignKey('customers.customerNumber'), nullable=False, index=True)

    customer = relationship('Customer')

In [ ]:
class Payment(Base):
    __tablename__ = 'payments'

    customerNumber = Column(ForeignKey('customers.customerNumber'), primary_key=True, nullable=False)
    checkNumber = Column(String(50), primary_key=True, nullable=False)
    paymentDate = Column(Date, nullable=False)
    amount = Column(DECIMAL(10, 2), nullable=False)

    customer = relationship('Customer')

In [ ]:
class Orderdetail(Base):
    __tablename__ = 'orderdetails'

    orderNumber = Column(ForeignKey('orders.orderNumber'), primary_key=True, nullable=False)
    productCode = Column(ForeignKey('products.productCode'), primary_key=True, nullable=False, index=True)
    quantityOrdered = Column(INTEGER(11), nullable=False)
    priceEach = Column(DECIMAL(10, 2), nullable=False)
    orderLineNumber = Column(SMALLINT(6), nullable=False)

    order = relationship('Order')
    product = relationship('Product')

In [ ]:
#Soal 1
result = session.query(Office).order_by(Office.country, Office.state, Office.city).with_entities(Office.country, Office.state, Office.city)
pd.read_sql(result.statement, engine)    

,country,state,city
0,Australia,None,Sydney
1,France,None,Paris
2,Japan,Chiyoda-Ku,Tokyo
3,UK,None,London
4,USA,CA,San Francisco
5,USA,MA,Boston
6,USA,NY,NYC


In [ ]:
#Soal 2
karyawan = session.query(Employee.employeeNumber).count()
print("Jumlah Karyawan yaitu :", karyawan)

Jumlah Karyawan yaitu : 23


In [ ]:
#Soal 3
payment = session.query(func.sum(Payment.amount)).scalar()
print("Total Pembayaran yaitu :",payment)

Total Pembayaran yaitu : 8853839.23


In [ ]:
#Soal 4
word_cars = session.query(Productline).filter(Productline.productLine.like('%Cars%'))
pd.read_sql(word_cars.statement, engine)

,productLine,textDescription,htmlDescription,image
0,Classic Cars,Attention car enthusiasts: Make your wildest c...,None,None
1,Vintage Cars,Our Vintage Car models realistically portray a...,None,None


In [ ]:
#Soal 5
pay_oct = session.query(func.sum(Payment.amount)).filter(Payment.paymentDate == '2004-10-28').scalar()
print("Total pembayaran pada 28 oktober 2004 yaitu : ", pay_oct) 

Total pembayaran pada 28 oktober 2004 yaitu :  47411.33


In [ ]:
#Soal 6
pay_more = session.query(Payment).filter(Payment.amount > 100000)
pd.read_sql(pay_more.statement, engine)

,customerNumber,checkNumber,paymentDate,amount
0,124,AE215433,2005-03-05,101244.59
1,124,KI131716,2003-08-15,111654.40
2,141,ID10962,2004-12-31,116208.40
3,141,JE105477,2005-03-18,120166.58
4,148,KM172879,2003-12-26,105743.00


In [ ]:
#Soal 7
line_product = session.query(Product).order_by(Product.productLine).with_entities(Product.productName, Product.productLine) 
pd.read_sql(line_product.statement, engine)

,productName,productLine
0,1976 Ford Gran Torino,Classic Cars
1,1958 Chevy Corvette Limited Edition,Classic Cars
2,1952 Citroen-15CV,Classic Cars
3,1982 Lamborghini Diablo,Classic Cars
4,1969 Chevrolet Camaro Z28,Classic Cars
...,...,...
105,1928 Ford Phaeton Deluxe,Vintage Cars
106,1911 Ford Town Car,Vintage Cars
107,1930 Buick Marquette Phaeton,Vintage Cars
108,1932 Model A Ford J-Coupe,Vintage Cars


In [ ]:
#Soal 8
product_count =func.count(Product.productCode)
result = session.query(Product.productLine ,product_count).group_by(Product.productLine).with_entities(Product.productName, product_count)
pd.read_sql(result.statement, engine)

,productName,count_1
0,1952 Alpine Renault 1300,38
1,1969 Harley Davidson Ultimate Chopper,13
2,1980s Black Hawk Helicopter,12
3,1999 Yamaha Speed Boat,9
4,Collectable Wooden Train,3
5,1958 Setra Bus,11
6,1937 Lincoln Berline,24


In [ ]:
#Soal 9
pay_min = session.query(func.min(Payment.amount)).scalar()
print("Jumlah minimum pembayaran :" ,pay_min)

Jumlah minimum pembayaran : 615.45


In [ ]:
#Soal 10
pay_avg = session.query(func.avg(Payment.amount)).scalar()
print("average payment : ", pay_avg)

pay_avg2 = pay_avg * 2
print("average payment 2 : ",pay_avg2)

result = session.query(Payment).filter(Payment.amount > pay_avg2)
pd.read_sql(result.statement, engine)

average payment :  32431.645531
average payment 2 :  64863.291062


,customerNumber,checkNumber,paymentDate,amount
0,114,MA765515,2004-12-15,82261.22
1,124,AE215433,2005-03-05,101244.59
2,124,BG255406,2004-08-28,85410.87
3,124,ET64396,2005-04-16,83598.04
4,124,KI131716,2003-08-15,111654.40
5,141,ID10962,2004-12-31,116208.40
6,141,IN446258,2005-03-25,65071.26
7,141,JE105477,2005-03-18,120166.58
8,148,KM172879,2003-12-26,105743.00
9,167,GN228846,2003-12-03,85024.46


In [ ]:
#Soal 11
avg_prctg = (Product.MSRP - Product.buyPrice) / Product.MSRP * 100
result = session.query(func.avg(avg_prctg))
pd.read_sql(result.statement, engine)

,avg_1
0,45.763505


In [ ]:
#Soal 12
dist_product = session.query(Product).distinct(Product.productName).count()
print("hasil :", dist_product)

hasil : 110


In [ ]:
#Soal 13
report_city = session.query(Customer).filter(Customer.salesRepEmployeeNumber == None).all()
for row in report_city:
        print(
        "Customer Name: ", row.customerName,
        "City: ", row.city
        )

Customer Name:  Havel & Zbyszek Co City:  Warszawa
Customer Name:  Porto Imports Co. City:  Lisboa
Customer Name:  Asian Shopping Network, Co City:  Singapore
Customer Name:  Natürlich Autos City:  Cunewalde
Customer Name:  ANG Resellers City:  Madrid
Customer Name:  Messner Shopping Network City:  Frankfurt
Customer Name:  Franken Gifts, Co City:  München
Customer Name:  BG&E Collectables City:  Fribourg
Customer Name:  Schuyler Imports City:  Amsterdam
Customer Name:  Der Hund Imports City:  Berlin
Customer Name:  Cramer Spezialitäten, Ltd City:  Brandenburg
Customer Name:  Asian Treasures, Inc. City:  Cork
Customer Name:  SAR Distributors, Co City:  Hatfield
Customer Name:  Kommission Auto City:  Münster
Customer Name:  Lisboa Souveniers, Inc City:  Lisboa
Customer Name:  Stuttgart Collectable Exchange City:  Stuttgart
Customer Name:  Feuer Online Stores, Inc City:  Leipzig
Customer Name:  Warburg Exchange City:  Aachen
Customer Name:  Anton Designs, Ltd. City:  Madrid
Customer Name

In [ ]:
#Soal 14
combine_name = session.query(func.concat(Employee.firstName," ", Employee.lastName)).filter(Employee.jobTitle.like('%VP%') | (Employee.jobTitle.like('%Manager%')))
for row in combine_name:
    print(
        row
    )

('Mary Patterson',)
('Jeff Firrelli',)
('William Patterson',)
('Gerard Bondur',)
('Anthony Bow',)


In [ ]:
#Soal 15
greater_than = session.query(Orderdetail).all()
for row in greater_than:
    i = row.quantityOrdered * row.priceEach
    if i > 5000:
        print(row.orderNumber, row.productCode)

10103 S10_1949
10103 S10_4962
10105 S10_4757
10105 S12_1108
10105 S24_2011
10108 S12_1099
10108 S18_4721
10108 S24_3856
10109 S18_1984
10109 S18_3232
10109 S18_3685
10110 S18_1749
10110 S18_2795
10110 S24_2887
10112 S10_1949
10114 S18_3232
10115 S12_4473
10115 S18_2238
10117 S12_1108
10117 S12_3148
10117 S12_3891
10119 S10_4757
10119 S18_1662
10120 S10_4698
10121 S12_2823
10122 S12_1099
10122 S24_3856
10123 S18_2870
10126 S10_1949
10126 S18_4600
10127 S12_1108
10127 S12_3148
10127 S12_3891
10127 S18_2238
10127 S18_2319
10127 S50_1392
10129 S24_2011
10134 S10_4698
10135 S12_1099
10135 S12_3380
10135 S18_3482
10135 S24_3856
10136 S18_3232
10137 S18_1589
10139 S18_2795
10140 S10_1949
10140 S24_2300
10141 S18_2238
10142 S12_1108
10142 S12_3891
10142 S18_3140
10143 S10_4757
10145 S10_4698
10145 S12_2823
10147 S12_1099
10148 S18_1589
10149 S18_1749
10150 S10_1949
10150 S18_4600
10151 S18_2238
10151 S18_2319
10153 S12_3148
10153 S12_3891
10155 S18_1662
10157 S700_2834
10159 S12_1099
10161 S18